In [18]:
# !pip install opencv-python-headless numpy matplotlib

# Import Libraries

In [19]:
import cv2
import numpy as np

# Load YOLO model

In [20]:
weights_path = '/content/yolov3-helmet.weights'
config_path = '/content/yolov3-helmet.cfg'
names_path = '/content/helmet.names'

# Function to load YOLOv3 model
def load_yolo_model(weights_path, config_path, names_path):
    net = cv2.dnn.readNet(weights_path, config_path)
    with open(names_path, 'r') as f:
        classes = [line.strip() for line in f.readlines()]
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
    return net, classes, output_layers

# Function to detect helmets in a frame

In [21]:
def detect_helmets(frame):
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)
    return outs

# Function to get bounding boxes from detections

In [22]:
def get_boxes(outs, height, width, conf_threshold=0.5):
    class_ids, confidences, boxes = [], [], []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > conf_threshold:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    return class_ids, confidences, boxes


# Function to draw labels on the frame

In [23]:
def draw_labels(frame, class_ids, confidences, boxes, classes):
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    font = cv2.FONT_HERSHEY_PLAIN
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = confidences[i]
            color = (0, 255, 0) if label == 'helmet' else (0, 0, 255)
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, f"{label} {confidence:.2f}", (x, y - 10), font, 1, color, 2)
    return frame

# Load the model
net, classes, output_layers = load_yolo_model(weights_path, config_path, names_path)


In [24]:
# Function to detect helmets in a video and save the output
def detect_from_video(video_path, output_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = None

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        height, width, _ = frame.shape

        # Initialize the video writer with the same dimensions and FPS as the input video
        if out is None:
            fps = cap.get(cv2.CAP_PROP_FPS)
            out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

        outs = detect_helmets(frame)
        class_ids, confidences, boxes = get_boxes(outs, height, width)
        frame = draw_labels(frame, class_ids, confidences, boxes, classes)
        out.write(frame)

    cap.release()
    if out is not None:
        out.release()
    cv2.destroyAllWindows()



In [ ]:
# Path to the uploaded video and the output video
video_path = '/content/vid-3.mp4'
output_path = '/content/processed_vid-3.mp4'

# Detect helmets in the video and save the output
detect_from_video(video_path, output_path)

# Display the processed video
from IPython.display import HTML
from base64 import b64encode

# Function to display video in Colab
def display_video(video_path):
    video_file = open(video_path, "rb").read()
    video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
    return HTML(f"""<video width=600 controls>
                      <source src="{video_url}" type="video/mp4">
                    </video>""")

# Display the processed video
display_video(output_path)


In [ ]:

from google.colab import files

# Download the processed video file
files.download('/content/processed_helmets_vid3.mp4')


In [ ]:
from google.colab import drive
drive.mount('/content/drive')